In [3]:
import tensorflow as tf
import keras
from keras import backend as K
from tensorflow.python.client import device_lib
import pandas as pd
import re
import numpy as np
import nltk
import requests
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from timeit import default_timer as timer
import math
import random
from collections import defaultdict
from keras.models import model_from_json
import h5py

Using TensorFlow backend.


Only run this code if you have a GPU. This part of the code makes the code run on your GPU, I used the tensorflow-gpu version 1.15 for this to work. It's considerably faster using my GPU (NVIDIA GTX 1060 6GB), than my CPU. About ~5 times faster, depending on the dataset I use.

In [4]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

print(device_lib.list_local_devices())
K.tensorflow_backend._get_available_gpus()

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:27:00.0, compute capability: 6.1

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11139154771864956303
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5083824128
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18103136064573924285
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:27:00.0, compute capability: 6.1"
]


['/job:localhost/replica:0/task:0/device:GPU:0']

In [48]:
df = pd.read_csv("brown.csv")
data = ' '.join(df['tokenized_text'].tolist()).replace("\'", "")
dataval = data[1518586:1700000]
data = data[0:1518586]

In [49]:
data[0:1000]

'Furthermore , as an encouragement to revisionist thinking , it manifestly is fair to admit that any fraternity has a constitutional right to refuse to accept persons it dislikes . The Unitarian clergy were an exclusive club of cultivated gentlemen -- as the term was then understood in the Back Bay -- and Parker was definitely not a gentleman , either in theology or in manners . Ezra Stiles Gannett , an honorable representative of the sanhedrin , addressed himself frankly to the issue in 1845 , insisting that Parker should not be persecuted or calumniated and that in this republic no power to restrain him by force could exist . Even so , Gannett judiciously argued , the Association could legitimately decide that Parker `` should not be encouraged nor assisted in diffusing his opinions by those who differ from him in regard to their correctness  . We today are not entitled to excoriate honest men who believed Parker to be downright pernicious and who barred their pulpits against his dem

In [8]:
df = pd.read_csv("jokes.csv")
data = ' '.join(df['Joke'].tolist()).replace("\'", "")
print(len(data))

128865


In [72]:
url = "https://www.gutenberg.org/files/46/46-0.txt"
book = requests.get(url)
data = book.text
data = data[1353:]
print(len(data))

180713


In [79]:
url = "http://gutenberg.org/files/1342/1342-0.txt"
book = requests.get(url)
data1 = book.text
data1 = data1[2440:]
print(len(data1))

797205


In [3]:
url = "http://gutenberg.org/files/65084/65084-0.txt"
book = requests.get(url)
data2 = book.text
data2 = data2[2974:]
data2 = data2[:-34443]
len(data2)

202023

In [53]:
url = "http://gutenberg.org/files/1250/1250-0.txt"
book = requests.get(url)
data3 = book.text
data3 = data3[1197:]
print(len(data3))

129658


In [40]:
url = "http://gutenberg.org/files/521/521-0.txt"
book = requests.get(url)
data4 = book.text
data4 = data4[1883:]
print(len(data4))

652062


In [96]:
data = data + data1 + data2 + data3 + data4

In [19]:
len(data)

1518586

In [6]:
data = data2

Cleaning of the data

In [12]:
def clean_dataset(dataset):

    dataset = ''.join(i for i in dataset if not i.isdigit())
    dataset = dataset.lower()
    dataset = dataset.replace("â", "")
    dataset = dataset.replace("_", "")
    dataset = dataset.replace("--", " ")
    dataset = dataset.replace("*", "")
    
    # remove whitespace
    tokenized = word_tokenize(dataset)
    dataset = " ".join([token.strip() for token in tokenized])

    # remove characters
    dataset = re.sub(r"[^\w\n]", " ", dataset)

    # replace multiple whitespaces with single whitespace
    dataset = re.sub(r"\s+", " ", dataset)
    dataset = dataset.strip()

    return dataset

data = clean_dataset(data)
dataval = clean_dataset(dataval)

In [14]:
data[0:1000]

'furthermore as an encouragement to revisionist thinking it manifestly is fair to admit that any fraternity has a constitutional right to refuse to accept persons it dislikes the unitarian clergy were an exclusive club of cultivated gentlemen as the term was then understood in the back bay and parker was definitely not a gentleman either in theology or in manners ezra stiles gannett an honorable representative of the sanhedrin addressed himself frankly to the issue in insisting that parker should not be persecuted or calumniated and that in this republic no power to restrain him by force could exist even so gannett judiciously argued the association could legitimately decide that parker should not be encouraged nor assisted in diffusing his opinions by those who differ from him in regard to their correctness we today are not entitled to excoriate honest men who believed parker to be downright pernicious and who barred their pulpits against his demand to poison the minds of their congre

In [15]:
# test set for measuring perplexity
test_num = int(0.9 * len(data))
testdata = data[test_num:len(data)]
data = data[0:test_num]

testdata = testdata.split()
testseqs = []

sequence_length = 4
seq_length = sequence_length

for i in range(sequence_length, len(testdata)):
    sequence = testdata[i-sequence_length:i+1]
    testseqs.append(sequence)

testseqs = np.array(testseqs)

XD = []
for x in testseqs:
    XD.append(' '.join(x))
    
testseqs = XD

# divide the sequence into data and target
testseqs = np.array(testseqs)

Xtest = []
Ytest = []
for sequence in testseqs:
    Xtest.append(' '.join(sequence.split()[0:-1]))
    Ytest.append(sequence.split()[-1])
    
print(Xtest[0])
print(Ytest[0])

ing and he thought
it


In [16]:
len(data)

1292564

In [17]:
# measure perplexity on train data
testdata = data[0:int((0.1*len(data)))]
testdata = testdata.split()
testseqs = []

for i in range(sequence_length, len(testdata)):
    sequence = testdata[i-sequence_length:i+1]
    testseqs.append(sequence)

testseqs = np.array(testseqs)

XD = []
for x in testseqs:
    XD.append(' '.join(x))
    
testseqs = XD

# divide the sequence into data and target
testseqs = np.array(testseqs)

Xtrainperplex = []
Ytrainperplex = []
for sequence in testseqs:
    Xtrainperplex.append(' '.join(sequence.split()[0:-1]))
    Ytrainperplex.append(sequence.split()[-1])
    
print(Xtrainperplex[0])
print(Ytrainperplex[0])

furthermore as an encouragement
to


In [18]:
#Tokenize train dataset
word_tokeniser = Tokenizer()
word_tokeniser.fit_on_texts([data])
words_embedding = word_tokeniser.texts_to_sequences([data])[0]
vocab_size = len(word_tokeniser.word_index) + 1
print("Vocabulary Size: ", vocab_size)

sequences = []

for i in range(sequence_length, len(words_embedding)):
    sequence = words_embedding[i-sequence_length:i+1]
    sequences.append(sequence)
sequences = np.array(sequences)

# divide the sequence into data and target
sequences = np.array(sequences)

X = sequences[:427736,:-1]  # assign all but last words of a sequence to X
y = sequences[:427736,-1]   # assign last word of each sequence to y
y = to_categorical(y, num_classes=vocab_size)
X = pad_sequences(X, maxlen=seq_length, padding='pre')

Vocabulary Size:  20023


In [20]:
# Create validation data
# Couldn't there be words in the validation data that hasn't been seen yet? What will happen?
# Also using vocab_size of train set, for correct input size
testdata = dataval

words_embedding1 = word_tokeniser.texts_to_sequences([testdata])[0]


sequences = []

for i in range(sequence_length, len(words_embedding1)):
    sequence = words_embedding1[i-sequence_length:i+1]
    sequences.append(sequence)
sequences = np.array(sequences)


# divide the sequence into data and target
sequences = np.array(sequences)

Xvalid = sequences[:227736,:-1]  # assign all but last words of a sequence to X
yvalid = sequences[:227736,-1]   # assign last word of each sequence to y
yvalid = to_categorical(yvalid, num_classes=vocab_size)
Xvalid = pad_sequences(Xvalid, maxlen=seq_length, padding='pre')

In [24]:

model = Sequential()
model.add(Embedding(vocab_size, 200, input_length = seq_length))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))

# output layer
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_crossentropy'])


In [25]:
def train_model(model, epochs, batch_size):
    
    epochsPer = int(epochs/10)

    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    json_file.close()

    for i in range(epochsPer, epochs+1, epochsPer):
        model.fit(X, y, epochs=epochsPer, verbose=1, batch_size=256)
        print("Loss and accuracy on valid")
        print(model.evaluate(x=Xvalid, y=yvalid, batch_size=128, verbose=0))
        

        model.save_weights(str(i) + "model.h5")
        print("Saved model to disk")
        print("")
    return model


        
model = train_model(model, 10, 256)

Epoch 1/1
226927/226927 [==============================] - 68s 299us/step - loss: 7.3339 - categorical_crossentropy: 7.3339
Loss and accuracy on valid
[6.760691001057835, 6.760693550109863]
Saved model to disk

Epoch 1/1
226927/226927 [==============================] - 76s 336us/step - loss: 6.7533 - categorical_crossentropy: 6.7533
Loss and accuracy on valid
[6.526186622404337, 6.526185512542725]
Saved model to disk

Epoch 1/1
226927/226927 [==============================] - 82s 360us/step - loss: 6.3997 - categorical_crossentropy: 6.3997
Loss and accuracy on valid
[6.44957383288698, 6.4495744705200195]
Saved model to disk

Epoch 1/1
  7424/226927 [..............................] - ETA: 59s - loss: 6.0524 - categorical_crossentropy: 6.0524 

KeyboardInterrupt: 

In [11]:
model.evaluate(x=Xvalid, y=yvalid, batch_size=128, verbose=0)

[8.39051994438104, 0.09809523820877075]

In [28]:
""""json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("decentmodel.h5")
print("Loaded model from disk")

Loaded model from disk


Make predictions using this function

In [29]:
def generate_words(model, word_tokeniser, seq_length, text, n_words):
    for x in range(n_words):
        
        # create word embeddings
        words_embedding = word_tokeniser.texts_to_sequences([text])[0]

        padded_words = pad_sequences([words_embedding], maxlen=seq_length, padding='pre')
        # predict next word
        print(padded_words)
        prediction = model.predict_classes(padded_words, verbose=0)

        print(sorted(model.predict(padded_words)[0], reverse=True)[0:5])

        next_word = ""
        for word, i in word_tokeniser.word_index.items():
            if [i] == prediction:
                next_word = word
                break
        text += " " + next_word
        
    return text

In [33]:

num_words = 1
i = random.randint(0,1000)
sentence = Xtrainperplex[i]
print(sentence)
print(Ytrainperplex[i])
print(generate_words(model, word_tokeniser, seq_length, sentence, num_words))

of their claim to
the
[[   2   40 1874    4]]
[0.2308572, 0.073507994, 0.069345936, 0.06533595, 0.05622109]
of their claim to come


In [45]:
def baseline_perplex(data):
    distribution = defaultdict(int)
    for word in data.split():
        distribution[word] += 1
        
    probdistribution = []   
    sumD = sum(distribution.values())
    for num in list(distribution.values()):
        probdistribution.append(num/sumD)
        
    count = 0
    for key in distribution.keys():
        distribution[key] = probdistribution[count]
        count += 1

    chance = 0
    print((1/sumD))
    for i in range(len(Ytest)):
        chance += math.log2(distribution.get(Ytest[i], (1/sumD)))
    return 2**((-1/len(Xtest))* chance)


baseline_perplex(data)

4.406625802556724e-06


1567.0408897674083

In [39]:
def perplex_help(model, word_tokeniser, seq_length, text, real_word):
        # create word embeddings
        words_embedding = word_tokeniser.texts_to_sequences([text])[0]
        # gets ID of the real word, and checks the probility assigned to it.
        # If the word hasn't been seen by the model before, I just make it 'the'
        padded_words = pad_sequences([words_embedding], maxlen=seq_length, padding='pre')
        idx = word_tokeniser.word_index.get(real_word, word_tokeniser.word_index['the'])
        prediction = model.predict_classes(padded_words, verbose=0)

        for word, i in word_tokeniser.word_index.items():
            if [i] == prediction:
                print(word)
        return model.predict(padded_words)[0][idx]

In [43]:
def perplexity(Xtest, Ytest):
    chance = 0
    for i in range(len(Xtest)):
        print("")
        print(Xtest[i])
        print(Ytest[i])
        wordprob = perplex_help(model, word_tokeniser, seq_length, Xtest[i], Ytest[i])
        if wordprob != 0:
            chance += math.log2(wordprob)
        
        print(wordprob)
    return 2**((-1/len(Xtest))* chance)

perplexity(Xtest, Ytest)


ing and he thought
it
entirely
5.1999343e-08

and he thought it
lovely
would
1.1070911e-20

he thought it lovely
he
knowledge
3.1282432e-06

thought it lovely he
didnt
guess
9.513626e-05

it lovely he didnt
doubt
feel
1.1404793e-17

lovely he didnt doubt
her
he
0.003692627

he didnt doubt her
truthfulness
he
1.36601575e-05

didnt doubt her truthfulness
although
she
2.1627971e-05

doubt her truthfulness although
he
they
0.046781205

her truthfulness although he
had
himself
0.034977753

truthfulness although he had
heard
become
0.001339142

although he had heard
the
so
0.009500773

he had heard the
words
elevator
0.00084658794

had heard the words
a
up
0.02494165

heard the words a
hundred
cent
2.090845e-05

the words a hundred
times
year
0.00034990546

words a hundred times
ive
district
5.7246884e-11

a hundred times ive
never
gone
0.0012558465

hundred times ive never
done
married
0.00021779229

times ive never done
this
she
0.008964911

ive never done this
before
would
1.5144275e-05


KeyboardInterrupt: 